In [1]:
import json
import logging
import sys
import os
import math
import shutil
from pathlib import Path
from itertools import chain
# from megatron.data.dataset_utils import get_indexed_dataset_

# import horovod.torch as hvd
# from dotenv import load_dotenv
import torch
import numpy as np
import datasets
from torch.utils.data import DataLoader, DistributedSampler
from datasets import Dataset, load_dataset
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score


import transformers  # noqa: E402
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser  # noqa: E402


sys.path.append('../')

from lm_experiments_tools.utils import collect_run_configuration, get_cls_by_name, get_optimizer  # noqa: E402
import lm_experiments_tools.optimizers as optimizers  # noqa: E402
# from lm_experiments_tools import TrainerArgs
import json
import logging
import sys
import os
import math
import random
import shutil
from pathlib import Path
from itertools import chain
from megatron.data.dataset_utils import get_indexed_dataset_

import horovod.torch as hvd
# from dotenv import load_dotenv
import torch
import numpy as np
import datasets
from torch.utils.data import DataLoader, DistributedSampler
from datasets import Dataset, load_dataset, load_from_disk
from huggingface_hub import hf_hub_download
from sklearn.metrics import f1_score, accuracy_score

from lm_experiments_tools import TrainerArgs
from lm_experiments_tools.trainer import Trainer

from torch.nn.utils.rnn import pad_sequence
from lm_experiments_tools.lm_datasets import get_lm_datasets



libgomp: Invalid value for environment variable OMP_NUM_THREADS


/home/jovyan/envs/accel_rmt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [2]:

# load_dotenv()

# logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#                     level=logging.INFO)
# logger = logging.getLogger(__name__)

# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# # if CUDA_VISIBLE_DEVICES is not set make all gpus visible
# if os.environ.get('CUDA_VISIBLE_DEVICES', None) is None:
#     os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in range(torch.cuda.device_count())])

# logger.info(f"CUDA_VISIBLE_DEVICES: {os.environ['CUDA_VISIBLE_DEVICES']}")
# # first call to torch.cuda.device_count() sets visible gpus, following calls will not change the result
# logger.info(f"CUDA DEVICE COUNT: {torch.cuda.device_count()}")

# hvd.init()


# # limit # of CPU threads to be used per pytorch worker, otherwise it might use all cpus and throttle gpus
# # > 2 fails cause of https://github.com/pytorch/pytorch/issues/56615
# # need to upgrade to torch>1.8.1
# torch.set_num_threads(4)
# # all gpus set with CUDA_VISIBLE_DEVICES are visible to process, indexing from 0 to ...
# torch.cuda.set_device(hvd.local_rank())

# parser = HfArgumentParser(TrainerArgs)
# parser.add_argument('--task_name', type=str, help="Task name, wikitext, ...")
# parser.add_argument('--validate_only', action='store_true', default=False,
#                     help='Skip training and run only validation. (default: False)')
# parser.add_argument('--working_dir', type=str, default='.',
#                     help='working dir, should be a dir with t5-experiments repo (default: .)')
# parser.add_argument('--seed', type=int, default=42, help='random seed')
# parser.add_argument('--show_valid_examples', type=int, default=0,
#                     help='how many valid examples to show during training (default: 0)')
# parser.add_argument('--input_seq_len', type=int, default=128, help='input sequnce length (default: 128).')
# parser.add_argument('--target_seq_len', type=int, default=16, help='target sequnce length, should be set to '
#                                                                    'max(len(target))+1 for EOS (default: 16).')
# parser.add_argument('--data_n_workers', type=int, default=2, help='number of dataloader workers (default: 2)')

# parser.add_argument('--input_prefix', type=str, default='', help='add task prefix to an input string (default: "")')
# parser.add_argument('--sliding_window', action='store_true', help='use slinding window attentinon mask, '
#                     'eval on last segment only', default=False)

# # model args
# parser.add_argument('--from_pretrained', type=str, help='model name in HF Model Hub (default: "")')
# parser.add_argument('--model_cfg', type=str, help='path to model configuration file (default: "")')
# parser.add_argument('--model_cls', type=str, default='transformers:BertForPreTraining',
#                     help='model class name to use (default: transformers:BertForPreTraining)')
# parser.add_argument('--memory_cell_cls', type=str, default=None, help='cell class for RMT')
# parser.add_argument('--recurrent_wrapper_cls', type=str, default=None, help='recurrent wrapper class for RMT')
# parser.add_argument('--model_cpt', type=str, default=None, help='pretrained model checkpoint path')
# parser.add_argument('--backbone_cls', type=str, default=None,
#                     help='backbone class name to use for RMT')
# parser.add_argument('--model_type', type=str, default='encoder-decoder',
#                     help='model type, encoder, encoder-decoder, decoder, affects preprocessing '
#                          '(default: encoder-decoder)')


# # Aydar # RMT args 
# parser.add_argument('--input_size', type=int, default=None, help='maximal input size of the backbone model')
# parser.add_argument('--num_mem_tokens', type=int, default=None, help='number of memory tokens.')
# parser.add_argument('--max_n_segments', type=int, default=1, help='maximal segment number')
# # parser.add_argument('--sum_loss', action='store_true', default=False,
# #                     help='with this flag task loss from all segments is summed')
# # parser.add_argument('--bptt_depth', type=int, default=-1, help='max number of previous segments in gradient computation.')
# # parser.add_argument('--segment_ordering', type=str, help='segment order', default='regular',
# #                     choices=['regular', 'reversed', 'bidirectional', 'repeat_first', 'last_memory_only'])
# # parser.add_argument('--memory_forward_func', type=str, help='path to memory forward funсtion script', default=None)
# # parser.add_argument('--memory_layers', type=str, help='memory-augmented layer inds or "all" for all layers', default=None)
# # parser.add_argument('--share_memory_layers', action='store_true', help='share weights of memory layers', default=False)
# # parser.add_argument('--reconstruction_loss_coef', type=float, default=None,
# #                     help='reconstuction loss ratio in total loss')
# # # parser.add_argument('--segment_ordering', type=str,help='????', default='regular',
# # #                     choices=['regular', 'reversed', 'bidirectional', 'repeat_first', 'last_memory_only'])
# # parser.add_argument('--retain_graph', action='store_true', help='Retain computation graph during backward pass', default=False)
# # parser.add_argument('--use_truncated_backward', action='store_true', default=False,
# #                     help='whether to use RMT truncated bptt method in backward')
# # parser.add_argument('--k1', type=int, default=-1, help='(not implemented) If not -1, gradient update is done each k1 segments')
# parser.add_argument('--k2', type=int, default=-1, help='number of last segments used by backward')
# parser.add_argument('--freeze_model_weights', action='store_true', default=False,
#                     help='Stop training all model weights except memory layers')
# parser.add_argument('--backbone_cpt', type=str, default=None, help='backbone model checkpoint path')


# # tokenizer
# # todo: add wordpiece tokenizers support?
# parser.add_argument('--tokenizer', type=str, default=None, help='path or name of pre-trained HF Tokenizer')

# # optimizer args
# parser.add_argument('--optimizer', type=str, default='AdamW', help='optimizer name: AdamW, Adafactor. (default: AdamW)')
# parser.add_argument('--weight_decay', type=float, default=0.0, help='optimizer weight decay (default: 0.0)')
# parser.add_argument('--scale_parameter', action='store_true', default=False,
#                     help='Adafactor scale_parameter (default: False)')
# parser.add_argument('--relative_step', action='store_true', default=False,
#                     help='Adafactor relative_step (default: False)')
# parser.add_argument('--warmup_init', action='store_true', default=False,
#                     help='Adafactor warmup_init (default: False)')

In [3]:
class Holder:
    def __init__(self):
        pass
    def __getattr__(self, name):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return None

args = Holder()

In [4]:
args.from_pretrained = 'gpt2'

In [5]:
attrs = ("--model_path", "/cephfs/home/bulatov/bulatov/RMT_light/runs/test/cnli_original",
"--from_pretrained", "gpt2",
"--task_name", "arxiv",
"--model_type", "decoder",
"--model_cls", "modeling_rmt.deprecated:RMTDecoderLMHeadMultiSeg",
# "--backbone_cls",  "transformers:AutoModelForCausalLM",
"--backbone_cls",  "base_models.modeling_gpt2:GPT2LMHeadModel",
"--model_cpt", "/home/jovyan/rmt/runs/arxiv/gpt2/linear_adamw_wd1e-03_868-128-7x128_mem2_bs32_regular_bptt-7_from_cpt_cv2_6-7/run_1", 
# "--model_cpt", "/home/jovyan/rmt/runs/arxiv/gpt2/linear_adamw_wd1e-03_248-128-2x128_mem2_bs32_regular_bptt-2_from_cpt_cv2_1-2/run_1", 
"--optimizer", "AdamW",
"--weight_decay", "0.001",
"--lr", "1e-03", 
"--lr_scheduler", "constant_with_warmup",
"--clip_grad_value", "5.0")

int_attrs = ("--input_seq_len", str((128-4)*64),
"--input_size", "128",
"--target_seq_len", "128",
"--num_mem_tokens", "2",
"--max_n_segments" ,"64", 
"--batch_size", "1", 
"--gradient_accumulation_steps", "1",
"--iters", "100",
"--num_warmup_steps", "100",
"--data_n_workers", "2",
"--log_interval", "10",
"--show_valid_examples", "5",
"--early_stopping_patience", "15",
"--seed", "42",
"--k2", "-1",)

In [6]:
for a, v in zip(int_attrs[::2], int_attrs[1::2]):
    setattr(args, a.split('--')[1], int(v))

for a, v in zip(attrs[::2], attrs[1::2]):
    try:
        setattr(args, a.split('--')[1], float(v))
    except ValueError:
        setattr(args, a.split('--')[1], v)

In [7]:

# args = parser.parse_args()
# set current working dir
# args.working_dir = str(Path(args.working_dir).expanduser().absolute())
# os.chdir(args.working_dir)
# if hvd.rank() == 0:
#     logger.info(f'hvd size: {hvd.size()}')
#     logger.info(f'FP16: {args.fp16}')

# if hvd.rank() == 0 and args.model_path is None:
#     logger.warning('model_path is not set: config, logs and checkpoints will not be saved.')

# # create model path and save configuration
# if hvd.rank() == 0 and args.model_path is not None:
#     model_path = Path(args.model_path)
#     if not model_path.exists():
#         Path(model_path).mkdir(parents=True)
#     args_dict = collect_run_configuration(args)
#     # todo: if model path exists and there is config file, write new config file aside
#     json.dump(args_dict, open(model_path/'config.json', 'w'), indent=4)

if not args.from_pretrained:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer)
else:
    tokenizer = AutoTokenizer.from_pretrained(args.from_pretrained)


In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
import random

block_size = args.input_size 
if args.num_mem_tokens is not None:
    block_size -= 2 * args.num_mem_tokens
if args.xl_cache_size is not None:
    block_size -= args.xl_cache_size
history_size = args.input_seq_len - block_size

class segmentDataLoaderOTF(DataLoader):
    def __init__(self, dataset, block_size, history_size, max_samples=None, shuffle=False, *args, **kwargs):
        super().__init__(dataset, *args, **kwargs)
        self.block_size = block_size
        self.history_size = history_size
        self.max_samples = max_samples
        self.shuffle = shuffle
            
    def get_samples(self, document):
        input_ids, attention_mask = document['input_ids'], document['attention_mask']
        samples = [input_ids[max({0, start - self.history_size}): start + self.block_size] for start in range(0, len(input_ids), self.block_size)]
        return samples
    
    def __iter__(self):
        inds = list(range(len(self.dataset)))

        if self.max_samples is not None:
            inds = inds[:self.max_samples]

        if self.shuffle: 
            np.random.shuffle(inds)

        doc_ind = 0
        samples = []

        while True:
            if doc_ind >= len(inds):
                break
            try: 
                while len(samples) < self.batch_size:
                    document = self.dataset[inds[doc_ind]]
                    doc_ind += 1
                    samples += self.get_samples(document)
                    if doc_ind >= len(inds):
                        raise(StopIteration)
            except(StopIteration):
                pass

            batch, samples = samples[:self.batch_size], samples[self.batch_size:]
            yield self.collate_fn(batch)
        

from torch.nn.utils.rnn import pad_sequence
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
def collate_fn(batch):
    input_ids = labels = [torch.tensor(b[::-1]) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
    labels = pad_sequence(labels, padding_value=-100).T.flip(1)
    attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask}
    
    if input_ids.shape[1] != block_size:
        labels_mask = torch.ones_like(input_ids, dtype=bool)
        labels_mask[:, :-block_size] = False
        collated['labels_mask'] = labels_mask

    if getattr(args, 'vary_n_segments', False):
        if getattr(args, 'sampling_prob', False):
            random_value = torch.rand((1)).item()
            if random_value > args.sampling_prob:
                return collated
        n_segments = random.randint(0, args.max_n_segments)
        n_tokens = n_segments * block_size
        for k in collated:
            collated[k] = collated[k][:, -n_tokens:]
    
    return collated

In [10]:
train_dataset = load_from_disk('/home/jovyan/rmt/datasets/arxiv/train')
valid_dataset = load_from_disk('/home/jovyan/rmt/datasets/arxiv/valid')
# test_dataset = load_from_disk('/home/jovyan/rmt/datasets/arxiv/test')

In [11]:

# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = RandomSampler(train_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=True,
                                    # drop_last=False, seed=args.seed)
per_worker_batch_size = args.batch_size 
# global_batch_size = per_worker_batch_size * hvd.size()
kwargs = {'pin_memory': True, 'num_workers': args.data_n_workers}
if not args.validate_only:
    train_dataloader = segmentDataLoaderOTF(train_dataset, batch_size=per_worker_batch_size, 
                                            # sampler=train_sampler,
                                    block_size=block_size, 
                                    history_size=history_size, 
                                    shuffle=True,
                                    collate_fn=collate_fn, **kwargs)
else:
    train_dataloader = None

# get validation dataset
max_samples = None if args.validate_only else 100
# max_samples = None
valid_dataloader = None
# if hvd.rank() == 0:
#     logger.info(f'preparing validation data')
# valid_sampler = DistributedSampler(valid_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=False)
valid_dataloader = segmentDataLoaderOTF(valid_dataset, batch_size=per_worker_batch_size,
                                        #  sampler=valid_sampler,
                                block_size=block_size, 
                                history_size=history_size, 
                                shuffle=False,
                                max_samples=max_samples,
                                collate_fn=collate_fn, drop_last=True, **kwargs)

In [12]:

# batch = next(gen)

In [13]:
# for batch in train_dataloader:
#     for b in batch['input_ids']:
#         if 50256 not in b:
#             break

In [14]:
gen = iter(train_dataloader)
batches = []

for _ in range(10000):
    batch = next(gen)
    inputs = batch['input_ids'][0]

    if len(inputs) >= args.input_seq_len and 50256 not in inputs:
        batches.append(inputs)

In [15]:
gen = iter(valid_dataloader)
valid_batches = []

for _ in range(10000):
    batch = next(gen)
    inputs = batch['input_ids'][0]

    if len(inputs) >= args.input_seq_len and 50256 not in inputs:
        valid_batches.append(inputs)

In [16]:
len(batches), len(valid_batches)

(5556, 5449)

In [17]:
# define model
model_cls = get_cls_by_name(args.backbone_cls)

[2023-09-20 09:24:41,638] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [18]:
import math
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

from modeling_rmt.deprecated import RMTBaseModel

class RMTDecoderLMHeadMultiSegSaveMem(RMTBaseModel):
    def set_params(self, num_mem_tokens, tokenizer, **rmt_config):
        self.rmt_config = rmt_config
        self.extract_special_tokens(tokenizer)
        self.create_memory(num_mem_tokens)

        self.segment_size = rmt_config['input_size'] - 2 * num_mem_tokens - tokenizer.num_special_tokens_to_add()
        if 'sep_token' in tokenizer.special_tokens_map:
            self.segment_size -= 1

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_weights = torch.randn((num_mem_tokens, self.model.config.n_embd)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        create_memory = True
        if create_memory:
            memory = self.memory.repeat(input_shape[0], 1, 1)
        else:
            memory = self.memory_state[:input_shape[0]]
        return memory
    
    def detach_memory(self, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
            or seg_num + k2 > max_n_segments:
            return False
        return True
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, labels=None, labels_mask=None, output_attentions=None, output_hidden_states=None, return_dict=None):
        kwargs = {'attention_mask': attention_mask, 
                #   'token_type_ids': token_type_ids,
                  'position_ids': position_ids, 'inputs_embeds': inputs_embeds,
                  'labels_mask': labels_mask, #'pos_weight': pos_weight,
                  'labels': labels, 'output_attentions': output_attentions,
                  'output_hidden_states': output_hidden_states, 'return_dict': return_dict,
                  }

        memory = self.set_memory(input_ids.shape)
        segmented = self.pad_and_segment(input_ids, labels)

        base_model_outputs = []
        memories = []
        for seg_num, segment in enumerate(zip(*segmented)):

            seg_kwargs, non_empty_mask = self.prepare_kwargs(segment, memory, kwargs)
            if sum(non_empty_mask) == 0:
                continue
            
            if self.detach_memory(seg_num):
                memory = memory.detach()

            seg_kwargs['inputs_embeds'][:, self.read_memory_position] = memory[non_empty_mask]
            seg_kwargs['inputs_embeds'][:, self.write_memory_position] = memory[non_empty_mask]
            out = self.model(**seg_kwargs)
            base_model_outputs.append(out)
            
            memory[non_empty_mask] = out.hidden_states[-1][:, self.write_memory_position]
            memories.append(memory[non_empty_mask])

        self.memory_state = memory
        out = self.process_outputs(base_model_outputs, kwargs)
        out['memories'] = memories
        return out
    
    def pad_and_segment(self, input_ids, labels=None):
        segmented_batch = []
        segmented_batch_labels = []

        if labels is None:
            labels = [None] * input_ids.shape[0]
        batch_labels = labels
        for seq, labels in zip(input_ids, batch_labels):

            align = self.rmt_config.get('segment_alignment')
            if align in {'right', None}:
                split_inds = (list(range(len(seq), 0, -self.segment_size)) + [0])[::-1]
            elif align == 'left':
                split_inds = list(range(0, len(seq), self.segment_size)) + [len(seq)]
            elif align == 'center':
                n_seg = math.ceil(len(seq) / self.segment_size)
                split_inds = list(range(0, len(seq), math.ceil(len(seq) / n_seg))) + [len(seq)]
            else:
                raise NotImplementedError
            input_segments = [seq[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
            # add empty segment markers if needed
            n_empty_segments = self.rmt_config['max_n_segments'] - len(input_segments)
            input_segments = [None] * n_empty_segments + input_segments
            segmented_batch.append(input_segments)

            if labels is not None:
                labels_segments = [labels[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
                labels_segments = [None] * n_empty_segments + labels_segments
                segmented_batch_labels.append(labels_segments)

        segmented_batch = [[sample[seg_num] for sample in segmented_batch]
                           for seg_num in range(self.rmt_config['max_n_segments'])]
        segmented_batch_labels = [[sample[seg_num] for sample in segmented_batch_labels]
                                  for seg_num in range(self.rmt_config['max_n_segments'])]

        return segmented_batch, segmented_batch_labels

    def prepare_kwargs(self, segment, memory, kwargs):
        segment_input_ids, segment_labels = segment
        seg_kwargs = dict(**kwargs)
        non_empty_mask = [s is not None for s in segment_input_ids]
        if sum(non_empty_mask) == 0:
            return None, non_empty_mask

        input_ids = torch.stack([s for s in segment_input_ids if s is not None])
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory, inputs_embeds, memory], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        seg_kwargs['attention_mask'] = self.get_attention_mask(inputs_embeds, input_ids)
        seg_kwargs['output_hidden_states'] = True
        if seg_kwargs['labels'] is not None:
            labels = torch.stack([el for el, m in zip(segment_labels, non_empty_mask) if m])
            memory_labels = torch.ones((labels.shape[0], self.num_mem_tokens), dtype=labels.dtype, device=labels.device) * -100
            seg_kwargs['labels'] = torch.cat((memory_labels, labels, memory_labels), dim=1)
            seg_kwargs['labels'][:, self.num_mem_tokens] = -100
        seg_kwargs.pop('labels_mask')

        return seg_kwargs, non_empty_mask
    
    def get_attention_mask(self, inputs_embeds, input_ids):
        pad = torch.ones(*inputs_embeds.shape[:2], dtype=torch.int64).to(inputs_embeds.device)
        if self.num_mem_tokens not in {0, None}:
            mask = input_ids != self.pad_token_id
            pad[:, self.num_mem_tokens: -self.num_mem_tokens] = mask
        return pad
    
    def process_outputs(self, model_outputs, kwargs):
        if self.num_mem_tokens in {0, None}:
            full_logits = torch.cat([o.logits for o in model_outputs], dim=1)
            truncated_hs = [[lh for lh in o.hidden_states] for o in model_outputs]
        else:    
            full_logits = torch.cat([o.logits[:, self.num_mem_tokens:-self.num_mem_tokens] for o in model_outputs], dim=1)
            truncated_hs = [[lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in o.hidden_states] for o in model_outputs]
        full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*truncated_hs)])

        rmt_out = CausalLMOutputWithCrossAttentions()
        full_labels = kwargs.get('labels')
        if full_labels is not None:
            shift_labels = full_labels[..., 1:].contiguous()
            shift_logits = full_logits[..., :-1, :].contiguous()
            flat_labels = shift_labels.view(-1)
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            
            loss_fct = CrossEntropyLoss()
            labels_mask = kwargs.get('labels_mask')
            if labels_mask is not None:
                shift_mask = labels_mask[..., :-1].contiguous()

                flat_labels = flat_labels[shift_mask.view(-1)]
                flat_logits = flat_logits[shift_mask.view(-1)]
                
                
            rmt_out['loss'] = loss_fct(flat_logits, flat_labels)

        rmt_out['logits'] = full_logits
        segment_keys = ['loss', 'logits']
        if kwargs.get('output_attentions'):
            segment_keys.append('attentions')
        if kwargs.get('output_hidden_states'):
            segment_keys.append('hidden_states')
            rmt_out['hidden_states'] = full_hidden_states

        for seg_num, out in enumerate(model_outputs):
            for key, value in out.items():
                if any([sk in key for sk in segment_keys]):
                    rmt_out[f'{key}_{seg_num}'] = value

        return rmt_out 

In [19]:
if not args.from_pretrained:
    model_cfg = AutoConfig.from_pretrained(args.model_cfg)
    model = model_cls(config=model_cfg)
else:
    # if hvd.rank() == 0:
    #     logger.info(f'Loading pretrained model: {args.from_pretrained}')
    model = model_cls.from_pretrained(args.from_pretrained)

## load cpt of backbone model
if args.backbone_cpt:
    backbone_cpt = os.path.join(args.backbone_cpt, "model_best.pth")
    cpt = torch.load(backbone_cpt, map_location='cpu')
    model.load_state_dict(cpt['model_state_dict'])
    # if hvd.rank() == 0:
    #     logger.info(f'Loaded baseline state dict from: {args.backbone_cpt}')
# Aydar # Pass memory settings to pretrained model
if args.num_mem_tokens is not None:
    if args.memory_forward_func is not None:
        args.memory_forward_func = get_cls_by_name(args.memory_forward_func)

    rmt_config = {
        'num_mem_tokens': args.num_mem_tokens, 
        'max_n_segments': args.max_n_segments,
        'xl_cache_size': args.xl_cache_size,
        # 'segment_ordering': args.segment_ordering,
        'input_size': args.input_size,
        'k1': args.k1, 'k2': args.k2,
        'sum_loss': args.sum_loss,
        'tokenizer': tokenizer,
        'memory_forward_func': args.memory_forward_func,
        'memory_layers': args.memory_layers,
        'share_memory_layers': args.share_memory_layers,
        'reconstruction_loss_coef': args.reconstruction_loss_coef,
    }
    # rmt_cls = get_cls_by_name(args.model_cls)
    rmt_cls = RMTDecoderLMHeadMultiSegSaveMem
    # if hvd.rank() == 0:
    #     logger.info(f'Wrapping in: {rmt_cls}')
    
    model = rmt_cls(model, **rmt_config)

    ## turn on memory resetting on validation
    model.rmt_config['reinit_mem_each_fwd'] = True

    ## load cpt of rmt
    if args.model_cpt:
        model_cpt = os.path.join(args.model_cpt, "model_best.pth")
        cpt = torch.load(model_cpt, map_location='cpu')
        model.load_state_dict(cpt['model_state_dict'])
        # if hvd.rank() == 0:
        #     logger.info(f'Loaded RMT state dict from: {args.model_cpt}')

In [20]:
# [k for k in model.state_dict().keys() if k not in cpt['model_state_dict'].keys()]
# [k for k in cpt['model_state_dict'].keys() if k not in model.state_dict().keys()]

2-64

In [21]:
1/0

ZeroDivisionError: division by zero

### What is in segments when memory is changed?

### Token 0

case 2

In [28]:
sample_idx = 1000
segment_idx = 31


batch = batches[sample_idx]
segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

n_seg = 8
n = tokenizer.decode(segments[0][segment_idx][0][:20]) + ' ... ' + tokenizer.decode(segments[0][segment_idx][0][-30:])
expl_str = "### Segment with different memory: {}".format(n)

for i in range(1, n_seg + 1):
    ni = tokenizer.decode(segments[0][segment_idx-i][0][:40]) + ' ... ' + tokenizer.decode(segments[0][segment_idx-i][0][-30:])
    expl_str += "\n\n\n### Segment n-{}: {}".format(i, ni)

print(expl_str)

### Segment with different memory:  the dimensionless form $${\boldsymbol{N}} = \left[ \begin{mat ... _1 &= \left[ \begin{matrix} 0            & -1 & \


### Segment n-1:  matrix and ${\boldsymbol{\eta}}$ is the Stroh vector, defined in. Note that the vector ${\boldsymbol{\eta}}$ is different from its counterpart in the ...  the charge-controlled case, the incremental electric boundary condition is in terms of the electric displacement $\Delta$. In the present situation the Stroh matrix has


### Segment n-2: $, $\bar{c}=c/\mu$ and $\bar{d}=d/\sqrt{\mu\varepsilon}$, and a prime denotes differentiation with respect ... mbol{N\eta}}, \label{stroh-form}$$ where ${\boldsymbol{N}}$ is the Stroh


### Segment n-3: \eta_5'=\mathrm{i}[(\bar{c}-\bar{a})\eta_2-\eta_4],
\notag \\
 ... 4],\end{aligned}$$ where $\bar{a}=a/\mu$, $\bar{b}=b/\mu


### Segment n-4:  \\
&\eta_2'=-\mathrm{i}\eta_1,
\notag \\
&\eta_3'=\mathrm{i}(2 ... }^2)\eta_1-\eta_5+2\bar{d}\eta_6],
\notag \\
&


### Segment n-5: ],\la

case 3

In [30]:
sample_idx = 1200
segment_idx = 31


batch = batches[sample_idx]
segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

n_seg = 9
n = tokenizer.decode(segments[0][segment_idx][0][:40]) + ' ... ' + tokenizer.decode(segments[0][segment_idx][0][-30:])
expl_str = "### Segment with different memory: {}".format(n)

for i in range(1, n_seg + 1):
    ni = tokenizer.decode(segments[0][segment_idx-i][0][:40]) + ' ... ' + tokenizer.decode(segments[0][segment_idx-i][0][-30:])
    expl_str += "\n\n\n### Segment n-{}: {}".format(i, ni)

print(expl_str)

### Segment with different memory:  c}
qb_1 & qb_2 & b_3 & b_4 \\
qb_5 & qb_6 & b_7 & b_8
 ... 7b_8}  = \dfrac{\tilde{g}-qb_1}{\tilde{g}-b


### Segment n-1: 6], which introduced the idea of the preservation of a connection matrix [@Birkhoff]. The article was also responsible for introducing a discrete analogue of the sixth Painlevé equation [@S ...  b_8
\end{array} : f,g \right\} \to \left\{\begin{array}{c c c


### Segment n-2: ,\lambda_1,\lambda_2}$ being trivial, we also have that the transformation $T_{a_1,a_2,a_3,\kappa_ ... 
-------------------------------------------------------------

This particular case was originally the subject of the article by Jimbo and Sakai [@Sakai:qP


### Segment n-3: end{gathered}$$ under the relations $$\begin{gathered}
a_1  = a_2b_3,\qquad
a_2  = a_ ... \kappa_2.\end{gathered}$$ In addition to $T_{\kappa_1,\kappa_2


### Segment n-4:  = \dfrac{q \left(y-a_1\right) \left(y-a_2\right) \kappa _1}{\left(y-a ... {a_2}{f},\qquad
z  = -\dfrac{b_2}{b_0}g,

case 4

In [32]:
sample_idx = 1400
segment_idx = 16


batch = batches[sample_idx]
segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

n_seg = 3
n = tokenizer.decode(segments[0][segment_idx][0][:20]) + ' ... ' + tokenizer.decode(segments[0][segment_idx][0][-30:])
expl_str = "### Segment with different memory: {}".format(n)

for i in range(1, n_seg + 1):
    ni = tokenizer.decode(segments[0][segment_idx-i][0][:40]) + ' ... ' + tokenizer.decode(segments[0][segment_idx-i][0][-30:])
    expl_str += "\n\n\n### Segment n-{}: {}".format(i, ni)

print(expl_str)

### Segment with different memory: (t,{\widehat{{\boldsymbol{x}}}}))\psi( ... }}))\psi(t)^{{d}/2}$, $i=1,\dots, {d}$.



### Segment n-1: hat{D}})}
+\dfrac{\lambda}{2}\int_0^T|{d_t}{{{\boldsymbol \alpha}}}|^2={\math ... {\widehat{{\boldsymbol{x}}}}):={{\boldsymbol P}}_i({\mathbf{X}}


### Segment n-2:  ${\mathcal{J}}$ as $$\label{eq:calJ_D}
{\mathcal{J}}({{{\boldsymbol \alpha}}})=\df ... -{\widehat{{\mathrm{f}}}}_i\|^2_{{\mathrm{L}}^2({\wide


### Segment n-3: t,{\mathbf{X}}(t,{\widehat{{\boldsymbol{x}}}}))\psi(t)^{{d}/2}\right)^ ... t,{\widehat{{\boldsymbol{x}}}}))=\psi(t)^{d}$. Then, we rewrite


### Token 1


case 3

In [ ]:
sample_idx = 1200
segment_idx = 34

In [ ]:
batch = batches[sample_idx]
segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

expl_str = "### Segment n-2: {}\n\n\n### Segment n-1: {}\n\n\n### Segment with different memory: {}"
n2 = tokenizer.decode(segments[0][segment_idx-2][0][:20]) + ' ... ' + tokenizer.decode(segments[0][segment_idx-2][0][-30:])
n1 = tokenizer.decode(segments[0][segment_idx-1][0][:20]) + ' ... ' + tokenizer.decode(segments[0][segment_idx-1][0][-30:])
n = tokenizer.decode(segments[0][segment_idx][0][:20]) + ' ... ' + tokenizer.decode(segments[0][segment_idx][0][-30:])

print(expl_str.format(n2, n1, n))

### Segment n-2:  $G_E^n (Q^2) = [1.91\tau/( ...  the matter distribution $G_0(Q)$ of the deuteron gives good agreement with the deuteron charge form factor defined by $


### Segment n-1: G_{ch}(Q) = 
G_0(Q)[G_E^ ...  fm and is smaller than that of the free proton radius. However, an input of the dipole form of $G_E^p


### Segment with different memory: $ reproduces the $ed$ data well as can also be found in [@arenhovel ... , $$\label{potd} 
V_d(Q) = 4 \pi e \, {G_0(Q)


case 2

In [ ]:
sample_idx = 1000
segment_idx = 27

In [ ]:
batch = batches[sample_idx]
segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

In [ ]:
expl_str = "### Segment n-2: ... {}\n\n\n### Segment n-1: ... {}\n\n\n### Segment with different memory: ... {}"
n2 = tokenizer.decode(segments[0][segment_idx-2][0][-60:])
n1 = tokenizer.decode(segments[0][segment_idx-1][0][-60:])
n = tokenizer.decode(segments[0][segment_idx][0][-80:])

print(expl_str.format(n2, n1, n))

### Segment n-2: ... int_{\Sigma_t} \left| \vec{H} + \frac{(x-y-(t_0-t)y')^\perp}{2(t_0-t)} \right|^2 \Phi_{y,t_0


### Segment n-1: ... _0} |y'(\tau)|^2 {\mathop{}\!\mathrm{d}}\tau\right) \int_{\Sigma_t} \Phi_{y(t),t_0}$ is non-increasing for $t<


### Segment with different memory: ...  The energy functional for curves in $\mathbb{R}^n$ is of course minimised by straight lines; in this case the equality case is easy to interpret:

Fix $x_0,y_0\in\mathbb{R}^n$ and $t_0\in\mathbb{R}$ and let $\{\Sigma_t\


In [ ]:
# segment_idx = 31
segment_idx = 35

In [ ]:
expl_str = "### Segment n-2: ... {}\n\n\n### Segment n-1: ... {}\n\n\n### Segment with different memory: ... {}"
n2 = tokenizer.decode(segments[0][segment_idx-2][0][-80:])
n1 = tokenizer.decode(segments[0][segment_idx-1][0][-80:])
n = tokenizer.decode(segments[0][segment_idx][0][-80:])

print(expl_str.format(n2, n1, n))

### Segment n-2: ... ^2\right).$$

We will now make use of the divergence theorem (\[eq:mcfdivthm\]). In particular, for fixed $t$ we set $$\label{eq:mcfX} X = -\frac{1}{2(t_0-t)}(x-y-2(t_0-t)y').


### Segment n-1: ...  X)&=& \Phi_{y,t_0}\left( {\operatorname{div}}_{\Sigma_t} X -  \frac{\langle X, (x-y)^T\rangle}{2(t_0-t)} \right)\\&=&\label{eq:mcfdiv} \Phi


### Segment with different memory: ... 2 - (t_0-t)^2|(y')^T|^2  }{4(t_0-t)^2}\right),\end{aligned}$$ where we have used that $x-y = (x-y-(t_0-t)y') + (t_0-t)y'$.

Combining


In [ ]:
segment_idx = 35

expl_str = "### Segment n-2: ... {}\n\n\n### Segment n-1: ... {}\n\n\n### Segment with different memory: ... {}"
n2 = tokenizer.decode(segments[0][segment_idx-2][0][-80:])
n1 = tokenizer.decode(segments[0][segment_idx-1][0][-80:])
n = tokenizer.decode(segments[0][segment_idx][0][-80:])

print(expl_str.format(n2, n1, n))

### Segment n-2: ... Q^2) Q^2/(8 m_p^2)]$ in [@gilmangross] (see Fig. 1). The proton radius corresponding to the dipole $G_E^p$ is 0.81 fm and is smaller than that of the free proton radius. However, an input of the dipole form of $G_E^p


### Segment n-1: ... $ interaction potential in (\[potqspace\]) by noting that the deuteron electric potential should be associated with the Coulomb interaction with form factors but cannot depend on the mass of the probe, in this case the muon. Thus, $$\label{potd} 
V_d(Q) = 4 \pi e \, {G_0(Q)


### Segment with different memory: ... ^2\over 8 m_N^2} \biggr ) \,,$$ where $e$ is the positive charge of the deuteron. Denoting, $G_0(Q)(G_E^p(Q^2) + G_E^n(Q^2)) (1 - Q^2/8m_N^2) = G_


case 1

In [ ]:
sample_idx = 0
# segment_idx = 23
segment_idx = 19

In [ ]:
expl_str = "### Segment n-2: ... {}\n\n\n### Segment n-1: ... {}\n\n\n### Segment with different memory: ... {}"
n2 = tokenizer.decode(segments[0][segment_idx-2][0][-30:])
n1 = tokenizer.decode(segments[0][segment_idx-1][0][-30:])
n = tokenizer.decode(segments[0][segment_idx][0][-30:])

print(expl_str.format(n2, n1, n))

### Segment n-2: ... ah\_omitting\_2014,goldbring\_enforceable\_2017,farah\_model\_2017]{


### Segment n-1: ...  focuses on the study of *classes* of objects, rather than single objects on their own. The important notion of *language* or *signature


### Segment with different memory: ...  $L$ has attached a natural number $n_{f}$, called its *arity*, and a function $%
\varpi ^


In [ ]:
# batch = batches[sample_idx]
# segments = model.pad_and_segment(input_ids=batch.reshape(1, -1))

# print("segment with different memory")
# print('... ' + tokenizer.decode(segments[0][segment_idx][0][-50:]))

# print()
# print()

# print("previous segment")
# print('... ' + tokenizer.decode(segments[0][segment_idx - 1][0][-50:]))

# print()
# print()

# print("previous segment")
# print('... ' + tokenizer.decode(segments[0][segment_idx - 2][0][-50:]))

### Save train memories

In [ ]:
res_dict = {}

for i in range(0, 2000, 200):
    batch = batches[i]
    collated = collate_fn([list(batch)])
    out = model(**collated)

    res_dict[f'sample_{i}_memories'] = out['memories']

for k in res_dict:
    res_dict[k] = [t.detach().numpy() for t in res_dict[k]]


In [ ]:
# path = '/home/jovyan/rmt/data/memories/7-64seg_memories_train.npy'
path = '/home/jovyan/rmt/data/memories/2-64seg_memories_train.npy'
np.save(path, res_dict,  allow_pickle=True)

### Save valid memories

In [ ]:
res_dict = {}

for i in range(0, 2000, 200):
    batch = valid_batches[i]
    collated = collate_fn([list(batch)])
    out = model(**collated)

    res_dict[f'sample_{i}_memories'] = out['memories']

for k in res_dict:
    res_dict[k] = [t.detach().numpy() for t in res_dict[k]]


In [ ]:
# path = '/home/jovyan/rmt/data/memories/7-64seg_memories_valid.npy'
path = '/home/jovyan/rmt/data/memories/2-64seg_memories_valid.npy'
np.save(path, res_dict,  allow_pickle=True)

7-64

### Save train memories

In [ ]:
res_dict = {}

for i in range(0, 2000, 200):
    batch = batches[i]
    collated = collate_fn([list(batch)])
    out = model(**collated)

    res_dict[f'sample_{i}_memories'] = out['memories']

for k in res_dict:
    res_dict[k] = [t.detach().numpy() for t in res_dict[k]]


In [ ]:
path = '/home/jovyan/rmt/data/memories/7-64seg_memories_train.npy'
# path = '/home/jovyan/rmt/data/memories/2-64seg_memories_train.npy'
np.save(path, res_dict,  allow_pickle=True)

### Save valid memories

In [ ]:
res_dict = {}

for i in range(0, 2000, 200):
    batch = valid_batches[i]
    collated = collate_fn([list(batch)])
    out = model(**collated)

    res_dict[f'sample_{i}_memories'] = out['memories']

for k in res_dict:
    res_dict[k] = [t.detach().numpy() for t in res_dict[k]]


In [ ]:
path = '/home/jovyan/rmt/data/memories/7-64seg_memories_valid.npy'
# path = '/home/jovyan/rmt/data/memories/2-64seg_memories_valid.npy'
np.save(path, res_dict,  allow_pickle=True)

In [ ]:
collate_fn([list(batches[0])])

{'input_ids': tensor([[6329,  198,  397,  ...,    3,  857,   13]]),
 'labels': tensor([[6329,  198,  397,  ...,    3,  857,   13]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]),
 'labels_mask': tensor([[False, False, False,  ...,  True,  True,  True]])}

In [ ]:
res_dicta

In [ ]:
out.loss

tensor(0.9173, grad_fn=<NllLossBackward0>)

In [ ]:
len(out.memories)

64

In [ ]:
out.memories[0]

In [ ]:
620981

In [ ]:

# define optimizer
optimizer_cls = get_optimizer(args.optimizer)
if optimizer_cls is None:
    raise RuntimeError(f'{args.optimizer} was not found in optimizers, torch.optim, transformers.optimization')

# if hvd.rank() == 0:
#     logger.info(f'Using optimizer class: {optimizer_cls}')

# todo: group optimizer params
if optimizer_cls in [transformers.optimization.Adafactor, optimizers.Adafactor]:
    # https://github.com/huggingface/transformers/pull/9751/files -> transformers 4.3.0
    optimizer = optimizer_cls(model.parameters(), lr=args.lr,
                                scale_parameter=args.scale_parameter,
                                relative_step=args.relative_step,
                                warmup_init=args.warmup_init,
                                weight_decay=args.weight_decay)
else:
    optimizer = optimizer_cls(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

def keep_for_metrics_fn(batch, output):
    data = {}
    if 'generation_outputs' in output:
            # data['labels'] = batch['answer']
        data['generation_outputs'] = output['generation_outputs']
        
    data['labels'] = batch['labels']
    for key in batch.keys():
        if 'loss' in key: 
            data[key] = batch[key]
    data['predictions'] = torch.argmax(output['logits'].detach(), dim=-1)
    if 'labels_mask' in batch:
        data['predictions'] = [data['predictions'][i, mask] for i, mask in enumerate(batch['labels_mask'])]
    return data

# HF datasets can compute metrics on each gpu process and then aggregate them on process with rank 0
# synchronization is done by using temporay files on a shared filesystem
# rank and number of workers is set by num_process and process_id params
# BUT our Trainer aggregates all prediction from all gpus!
#   this will lead to computing metrics for predictions repeated xN_GPUS times
# need to try:
# - keep_in_memory=True, may lead to OOM for large validation sets, after sync predictions and targets for the full
#       validation set would be stored on each GPU -> xN_GPUs RAM
#   - implemented currently
# - compute metrics on batch lvl
# - add support of HF metrics and turn off aggregation in case if metric has .add_batch method
# scrolls_metric = datasets.load_metric(scrolls_metric_path, args.task_name, keep_in_memory=True)

def metrics_fn(data):
    metrics = {}
    y, p = None, None
    if 'generation_outputs' in data:
        # replace -100 with pad token in labels
        y = data['labels']
        p = tokenizer.batch_decode(data['generation_outputs'], skip_special_tokens=True)
        
        if hvd.rank() == 0 and args.show_valid_examples > 0:
            for i in range(min(args.show_valid_examples, len(y))):
                logger.info(f'y: {y[i]}')
                logger.info(f'p: {p[i]}')
                logger.info(f'p ids: {data["generation_outputs"][i]}')
                logger.info('-' * 50)

    if y is not None and p is not None:
        metrics['exact_match'] = accuracy_score(y, p) * 100

    return metrics


In [ ]:
train_dataset[0]

{'context': "Receiving Party shall not solicit some of Disclosing Party's representatives. MUTUAL NON-DISCLOSURE AGREEMENT\nThis Non-Disclosure Agreement (“Agreement”) is made on ___ day of ___ 20__, (“effective date”) by and between _____________________, a _____________ Corporation, (“XXXXX”) and Data Boiler Technologies, LLC., a Massachusetts corporation (“DBT”).\nWHEREAS, DBT and XXX (the “Parties”) desire to engage in business related discussions and negotiations regarding a potential business relationship (“Transaction”).\nWHEREAS, the Parties may provide to each other certain confidential and proprietary information in connection with the Transaction and each desires that any such information provided shall be kept confidential by the other party; and\nWHEREAS, in consideration of the disclosure of such information, each party is willing to keep the other party’s information confidential in accordance with the terms and conditions set forth in this Agreement;\nNOW, THEREFORE, DB

In [ ]:
# gen = iter(train_dataloader)
# batch = next(gen)

batch_train = collate_train([train_dataset[i] for i in range(4)])
batch_valid = collate_valid([valid_dataset[i] for i in range(4)])

In [ ]:
# gen = iter(valid_dataloader)
# batch = next(gen)

In [ ]:
batch_train

{'input_ids': tensor([[ 3041,   344,  1412,  ...,  1463,   276, 50256],
         [18546, 35599,  6188,  ...,  6335,  2867, 50256],
         [ 3041,   344,  1412,  ...,   603,   434, 50256],
         [ 3041,   344,  1412,  ...,   603,   434, 50256]]),
 'labels': tensor([[ 3041,   344,  1412,  ...,  1463,   276, 50256],
         [18546, 35599,  6188,  ...,  6335,  2867, 50256],
         [ 3041,   344,  1412,  ...,   603,   434, 50256],
         [ 3041,   344,  1412,  ...,   603,   434, 50256]]),
 'labels_mask': tensor([[False, False, False,  ...,  True,  True, False],
         [False, False, False,  ...,  True,  True, False],
         [False, False, False,  ...,  True,  True, False],
         [False, False, False,  ...,  True,  True, False]]),
 'attention_mask': tensor([[ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True, Fa

In [ ]:
tokenizer.decode(batch_train['input_ids'][0])

"Receiving Party shall not solicit some of Disclosing Party's representatives. MUTUAL NON-DISCLOSURE AGREEMENT\nThis Non-Disclosure Agreement (“Agreement”) is made on ___ day of ___ 20__, (“effective date”) by and between _____________________, a _____________ Corporation, (“XXXXX”) and Data Boiler Technologies, LLC., a Massachusetts corporation (“DBT”).\nWHEREAS, DBT and XXX (the “Parties”) desire to engage in business related discussions and negotiations regarding a potential business relationship (“Transaction”).\nWHEREAS, the Parties may provide to each other certain confidential and proprietary information in connection with the Transaction and each desires that any such information provided shall be kept confidential by the other party; and\nWHEREAS, in consideration of the disclosure of such information, each party is willing to keep the other party’s information confidential in accordance with the terms and conditions set forth in this Agreement;\nNOW, THEREFORE, DBT and XXX he

In [ ]:
tokenizer.decode(batch_train['input_ids'][0][batch_train['labels_mask'][0]])

'[GEN]NotMentioned'

In [ ]:
batch_valid

{'input_ids': tensor([[ 3041,   344,  1412,  ...,   287,   393, 50257],
         [18546, 35599,  6188,  ...,   286, 50257, 50256],
         [ 4366, 13675,   286,  ..., 33147,  1626, 50257],
         [ 3041,   344,  1412,  ...,   290, 50257, 50256]]),
 'labels': tensor([[14539,   603,   434, 50256, 50256],
         [ 3673,    44,  1463,   276, 50256],
         [14539,   603,   434, 50256, 50256],
         [ 3673,    44,  1463,   276, 50256]]),
 'attention_mask': tensor([[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True, False],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True, False]])}

In [ ]:
tokenizer.decode(batch_valid['input_ids'][0])

"Receiving Party may share some Confidential Information with some of Receiving Party's employees. Attachment 8\nNon-Disclosure Agreement for Request for Proposal – California Courts Protective Order Registry Development\nTHIS CONFIDENTIALITY AGREEMENT (“Agreement”), effective as of _________________[date of company’s signed proposal] (“Effective Date”), is entered into by and between the Administrative Office of the Courts (the “AOC”) and [company name], a [ ] corporation, with its principal address at [complete address], (hereinafter the “Company”).\nWHEREAS, the State of California, acting through the Judicial Council of California, Administrative Office of the Courts is planning to disclose certain confidential information to potential vendors who wish to bid on the Request for Proposal for Judicial Council of California Administrative Office of the Courts (AOC) for California Courts Protective Order Registry Development, including all Statements of Work, dated on or about April, 2

In [ ]:
tokenizer.decode(batch_valid['labels'][0])

'Entailment<|endoftext|><|endoftext|>'

In [ ]:
labels = batch.pop('labels')

In [ ]:
generated = model.memory_cell.model.generate(**batch, max_new_tokens=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.batch_decode(generated)

["Receiving Party may share some Confidential Information with some of Receiving Party's employees. Attachment 8\nNon-Disclosure Agreement for Request for Proposal – California Courts Protective Order Registry Development\nTHIS CONFIDENTIALITY AGREEMENT (“Agreement”), effective as of _________________[date of company’s signed proposal] (“Effective Date”), is entered into by and between the Administrative Office of the Courts (the “AOC”) and [company name], a [ ] corporation, with its principal address at [complete address], (hereinafter the “Company”).\nWHEREAS, the State of California, acting through the Judicial Council of California, Administrative Office of the Courts is planning to disclose certain confidential information to potential vendors who wish to bid on the Request for Proposal for Judicial Council of California Administrative Office of the Courts (AOC) for California Courts Protective Order Registry Development, including all Statements of Work, dated on or about April, 

In [ ]:
tokenizer.batch_decode(labels)

['Entailment<|endoftext|><|endoftext|>', 'NotMentioned<|endoftext|>']

In [ ]:
self = model
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']

In [ ]:
memory_state = None
segmented = self.segment(input_ids=input_ids, attention_mask=attention_mask)

cell_outputs = []
for seg_num, segment in enumerate(segmented[:-1]):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    cell_outputs.append(cell_out)
    # self.manage_gradients(memory_state, seg_num)

final_segment = segmented[-1]
out = self.memory_cell.generate(**final_segment, memory_state=memory_state)

ModuleAttributeError: 'MemoryCell' object has no attribute 'generate'

In [ ]:
kwargs = dict(**final_segment)
input_ids = kwargs.pop('input_ids')

In [ ]:
seg_kwargs = self.memory_cell.process_input(input_ids, memory_state, **kwargs)
out = self.memory_cell.model.generate(**seg_kwargs)
# out, new_memory_state = self.memory_cell.process_output(out, **kwargs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: If inputs_embeds is passed as model-specific keyword input then model has to be an encoder-decoder and not a GPT2LMHeadModel.

In [ ]:
seg_kwargs['inputs_embeds'].shape

torch.Size([2, 16, 768])

In [ ]:
model.memory_cell.model.generate(seg_kwargs['inputs_embeds'], max_new_tokens=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: shape '[-1, 768]' is invalid for input of size 32

In [ ]:
len(cell_outputs), len(segmented)

(4, 5)

In [ ]:
len

In [ ]:
batch['input_ids'].shape

torch.Size([2, 508])

In [ ]:
tokenizer.batch_decode(batch['input_ids'])

["Receiving Party may share some Confidential Information with some of Receiving Party's employees. Attachment 8\nNon-Disclosure Agreement for Request for Proposal – California Courts Protective Order Registry Development\nTHIS CONFIDENTIALITY AGREEMENT (“Agreement”), effective as of _________________[date of company’s signed proposal] (“Effective Date”), is entered into by and between the Administrative Office of the Courts (the “AOC”) and [company name], a [ ] corporation, with its principal address at [complete address], (hereinafter the “Company”).\nWHEREAS, the State of California, acting through the Judicial Council of California, Administrative Office of the Courts is planning to disclose certain confidential information to potential vendors who wish to bid on the Request for Proposal for Judicial Council of California Administrative Office of the Courts (AOC) for California Courts Protective Order Registry Development, including all Statements of Work, dated on or about April, 

In [ ]:
batch['attention_mask']

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True, False]])

In [ ]:
tokenizer.batch_decode(batch['labels'])

['Entailment<|endoftext|><|endoftext|>', 'NotMentioned<|endoftext|>']

In [ ]:
cpt = torch.load("../../runs/cnli_original/gpt2/lr5e-05_linear_adamw_wd1e-03_124-128-1x128_mem2_bs32_iters6000_regular_bptt-1_refactor/run_1/model_best.pth", map_location='cpu')
model.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>

In [ ]:
# for i in range(10):
#     out = model(**batch)
#     out.loss.backward()
#     optimizer.step()

In [ ]:
batch['input_ids'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276])

In [ ]:
batch = next(gen)

In [ ]:
out = model(**batch)

In [ ]:
out.loss

tensor(1.3551, grad_fn=<NllLossBackward>)

In [ ]:
pred = out.logits.argmax(dim=-1)


In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['Notailment<|endoftext|>', 'Notailment<|endoftext|>']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(batch['labels'], batch['labels_mask'])], add_special_tokens=False)

['[GEN]Entailment', '[GEN]Entailment']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['NotMentioned<|endoftext|>', 'Notailment<|endoftext|>']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['NotMentioned<|endoftext|>', 'NotMentionedNot']

In [ ]:
out.loss

tensor(0.2207, grad_fn=<NllLossBackward>)

In [ ]:
gen_inputs = [b[~m] for b, m in zip(batch['input_ids'], batch['labels_mask'])]

In [ ]:
tokenizer.decode(gen_inputs[0])

'Receiving Party shall not disclose the fact that Agreement was agreed or negotiated. Johns Hopkins University\nNON-DISCLOSURE AGREEMENT For Bilateral Disclosure\nThis Agreement is effective this of in the year ______ is by and between JHU and COMPANY, each defined below.\nJHU: The Johns Hopkins University\nAddress: 100 N. Charles St., 5th Floor\nBaltimore, Maryland 21201\nJHU Contact:\nCOMPANY:\nAddress:\nCOMPANY Contact:\nWHEREAS, each party has certain technical information described below which shall hereinafter be referred to as "CONFIDENTIAL INFORMATION";\nCONFIDENTIAL INFORMATION:\nWHEREAS, JHU and COMPANY are each interested in examining the CONFIDENTIAL INFORMATION of the other solely for the PURPOSE, defined below;\nPURPOSE: To explore licensing, collaborative or sponsored research agreement opportunities related to the CONFIDENTIAL INFORMATION.\nNOW, THEREFORE, in consideration of the premises and mutual covenants contained herein, the parties hereto agree as follows:\n”PROV

In [ ]:
gt = tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

In [ ]:
gt

['Entailment', 'Entailment']

In [ ]:
accuracy_score(pred, pred)

1.0

In [ ]:
tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

['NotMentioned', 'Contradiction']

In [ ]:
out.loss

tensor(62.5257, grad_fn=<NllLossBackward>)

In [ ]:
for i in range(15):
    out = model(**batch)
    out.loss.backward()
    optimizer.step()

In [ ]:
out.loss

tensor(7.8596, grad_fn=<NllLossBackward>)

In [ ]:
pred = out.logits.argmax(dim=-1)

In [ ]:
pred_text = tokenizer.batch_decode(pred)

In [ ]:
generated = model.memory_cell.model.generate(batch['labels'][:, -5:])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
batch['input_ids'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

In [ ]:
batch['labels'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

In [ ]:
gen_inputs = [torch.cat((b[~m], torch.tensor([gen_token]))) for b, m in zip(batch['input_ids'], batch['labels_mask'])]

In [ ]:
gen = model.memory_cell.model.generate(gen_inputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
output = generated

In [ ]:
 def keep_for_metrics_fn(batch, output):
    data = {}
    if isinstance(output, torch.Tensor):
        data['labels'] = batch['labels']
        
    for key in batch.keys():
        if 'loss' in key: 
            data[key] = batch[key]
    return data

In [ ]:
tokenizer.batch_decode(generated, add_special_tokens=False)

['[GEN]NotMentioned[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]',
 ' in[GEN]Contradiction[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]']

In [ ]:
#  def keep_for_metrics_fn(batch, output):
data = {}
# if 'generation_outputs' in output:
if isinstance(output, dict):
    data['predictions'] = torch.argmax(output['logits'].detach(), dim=-1)
    if 'labels_mask' in batch:
        data['predictions'] = [data['predictions'][i, mask] for i, mask in enumerate(batch['labels_mask'])]
else:
    data['generation_outputs'] = output
    
data['labels'] = batch['labels']
for key in batch.keys():
    if 'loss' in key: 
        data[key] = batch[key]

In [ ]:
# out = CausalLMOutputWithCrossAttentions()
# full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
# full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

full_logits = out.logits
labels = kwargs.get('labels')
if labels is not None:
    shift_labels = labels[..., 1:].contiguous()
    shift_logits = full_logits[..., :-1, :].contiguous()
    flat_labels = shift_labels.view(-1)
    flat_logits = shift_logits.view(-1, shift_logits.size(-1))
    
    loss_fct = CrossEntropyLoss()
    labels_mask = kwargs.get('labels_mask')
    if labels_mask is not None:
        shift_mask = labels_mask[..., :-1].contiguous()

        flat_labels = flat_labels[shift_mask.view(-1)]
        flat_logits = flat_logits[shift_mask.view(-1)]
        
    out['loss'] = loss_fct(flat_logits, flat_labels)

out['logits'] = full_logits
segment_keys = ['loss', 'logits']
if kwargs.get('output_attentions'):
    segment_keys.append('attentions')
if kwargs.get('output_hidden_states'):
    segment_keys.append('hidden_states')
    out['hidden_states'] = full_hidden_states

In [ ]:

### booydar
batch_metrics_fn = lambda _, y: {key: y[key] for key in y.keys() if (('loss' in key) or ('!log' in key))}
trainer = Trainer(args, model, optimizer, train_dataloader, valid_dataloader, train_sampler,
                    keep_for_metrics_fn=keep_for_metrics_fn, metrics_fn=metrics_fn,
                    ###booydar
                    batch_metrics_fn=batch_metrics_fn,
                    generate_kwargs={})

if not args.validate_only:
    # train loop
    trainer.train()
    # make sure all workers are done
    hvd.barrier()
    # run validation after training
    if args.save_best:
        best_model_path = str(Path(args.model_path) / 'model_best.pth')
        if hvd.rank() == 0:
            logger.info(f'Loading best saved model from {best_model_path}')
        trainer.load(best_model_path)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Runnning validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=False)
else:
    # run validation, do not write to tensorboard
    if hvd.rank() == 0:
        logger.info('Running validation on train set:')
    trainer.validate(train_dataloader, split='train', write_tb=True)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Running validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=True)
    # if test_dataloader is not None:
    #     if hvd.rank() == 0:
    #         logger.info('Runnning validation on test data:')
    #     trainer.validate(test_dataloader, write_tb=True)


NameError: name 'train_sampler' is not defined

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Finetune

In [ ]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import sys
sys.path.append('..')

### load dataset 

In [ ]:
class Holder:
    def __init__(self):
        pass

In [ ]:
input_seq_len = 512
target_seq_len = 512

num_mem_tokens = 2
input_size = 128

batch_size = 2

args = Holder
args.target_seq_len = target_seq_len
args.input_seq_len = input_seq_len
args.num_mem_tokens = num_mem_tokens
args.input_size = input_size
args.input_prefix = ''
args.block_size = None
args.task_name = 'wikitext-2-v1'

device = 'cpu'

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
from itertools import chain

raw_datasets = datasets.load_dataset('wikitext', args.task_name)
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)

block_size = args.input_size 
if args.num_mem_tokens is not None:
    block_size -= 2 * args.num_mem_tokens
history_size = args.input_seq_len - block_size

def group_texts(examples, block_size, history_size=None):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if history_size is None:
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    else:
        result = {
            k: [t[max({0, i - history_size}) : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result

Found cached dataset wikitext (/home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-389b922bfc5fe729.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6067a66e735cfbb1.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-941845a5470f2db7.arrow


In [ ]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

In [ ]:
block_size, history_size

(124, 388)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     input_ids = [torch.tensor(b['input_ids'][::-1]) for b in batch]
#     labels = [torch.tensor(b['labels'][::-1]) for b in batch]
#     attention_mask = [torch.tensor(b['attention_mask'][::-1]) for b in batch]
#     input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
#     labels = pad_sequence(labels, padding_value=-100).T.flip(1)
#     attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

#     collated = {'input_ids': input_ids,
#                 'labels': labels, 
#                 'attention_mask': attention_mask}

#     if input_ids.shape[1] != block_size:
#         labels_mask = torch.ones_like(input_ids, dtype=bool)
#         labels_mask[:, :-block_size] = False
#         collated['labels_mask'] = labels_mask

#     return collated


def collate_fn(batch):
    input_ids = [torch.tensor(b['input_ids']) for b in batch]
    labels = [torch.tensor(b['labels']) for b in batch]
    labels_mask = [torch.ones_like(l, dtype=bool) for l in labels]
    attention_mask = [torch.tensor(b['attention_mask']) for b in batch]

    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T
    labels = pad_sequence(labels, padding_value=-100).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T
    attention_mask = pad_sequence(attention_mask, padding_value=0).T

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'labels_mask': labels_mask,
                'attention_mask': attention_mask}

    # if args.vary_n_segments:
    #     n_segments = np.random.randint(1, args.max_n_segments + 1)
    #     n_tokens = n_segments * block_size
    #     for k in collated:
    #         collated[k] = collated[k][:, -n_tokens:]

    return collated


train_dataset = tokenized_datasets["train"].map(lambda x: group_texts(x, block_size, history_size), 
                                        batched=True, desc=f"Grouping train in chunks of {block_size} and history {history_size}")
valid_dataset = tokenized_datasets["validation"].map(lambda x: group_texts(x, block_size), 
                                        batched=True, desc=f"Grouping valid in chunks of {block_size}")


# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedStrain_dataset[i] for i in range(4)ampler(train_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=True,
#                                     drop_last=False, seed=args.seed)
# per_worker_batch_size = args.batch_size * args.gradient_accumulation_steps
# global_batch_size = per_worker_batch_size * hvd.size()

# train_sampler = RandomSampler(train_dataset)
kwargs = {'pin_memory': True}#, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, **kwargs)

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1504f9373e317eca.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-c6da793e710ea6d8.arrow


In [ ]:
b = [train_dataset[i] for i in range(4)]

In [ ]:
for k in b[0]:
    b[0][k] = b[0][k][:124]

In [ ]:
batch = collate_fn(b)

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 512])

In [ ]:
# raw_datasets['train'][1]

### Model

In [ ]:
import math
import torch
from torch.nn import CrossEntropyLoss

In [ ]:
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class MemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens):
        super().__init__()
        self.model = base_model
        self.create_memory(num_mem_tokens)

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_weights = torch.randn((num_mem_tokens, self.model.config.n_embd)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        memory = self.memory.repeat(input_shape[0], 1, 1)
        return memory

    def forward(self, input_ids, memory_state=None, **kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, **kwargs)
        out = self.model(**seg_kwargs)
        out, new_memory_state = self.process_output(out, **kwargs)

        return out, new_memory_state
    
    def process_input(self, input_ids, memory_state, **kwargs):
        seg_kwargs = dict(**kwargs)

        inputs_embeds = kwargs.get('inputs_embeds')
        if inputs_embeds is None:
            inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory_state, inputs_embeds, memory_state], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        if kwargs.get('attention_mask') is not None:
            seg_kwargs['attention_mask'] = self.pad_attention_mask(kwargs['attention_mask'], inputs_embeds.shape)
        seg_kwargs['output_hidden_states'] = True
        return seg_kwargs
    
    def pad_attention_mask(self, attention_mask, shape):
        if self.num_mem_tokens in {0, None}:
            return attention_mask
        else:
            mask = torch.ones(*shape[:2], dtype=torch.int64).to(attention_mask.device)
            mask[:, self.num_mem_tokens:-self.num_mem_tokens] = attention_mask
            return mask
    
    def process_output(self, model_outputs, **kwargs):
        if self.num_mem_tokens not in {0, None}:
            out = CausalLMOutputWithCrossAttentions()
            memory_state = model_outputs.hidden_states[-1][:, -self.num_mem_tokens:]
            out['logits'] = model_outputs.logits[:, self.num_mem_tokens:-self.num_mem_tokens]
            
            if kwargs.get('output_hidden_states'):
                out['hidden_states'] = [lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in model_outputs.hidden_states]
            if kwargs.get('output_attentions'):
                out['attentions'] = model_outputs['attentions']
        else:
            memory_state = None
            out = model_outputs
            
        return out, memory_state 


In [ ]:
class RecurrentWrapper(torch.nn.Module):
    def __init__(self, memory_cell, **rmt_kwargs):
        super().__init__()
        self.memory_cell = memory_cell
        self.rmt_config = rmt_kwargs

    def forward(self, input_ids, labels=None, labels_mask=None, inputs_embeds=None, attention_mask=None, output_attentions=None, output_hidden_states=None):
        memory_state = None
        segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

        cell_outputs = []
        for seg_num, segment in enumerate(segmented):
            cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
            cell_outputs.append(cell_out)
            self.manage_gradients(memory_state, seg_num)

        out = self.process_outputs(cell_outputs, labels=labels, 
                                   labels_mask=labels_mask,
                                   output_attentions=output_attentions, 
                                   output_hidden_states=output_hidden_states)
        return out

    def segment(self, **kwargs):
        segments = []
        for k, tensor in kwargs.items():
            if tensor is not None:
                k_segments = self.split_tensor(tensor)
                for s, k_seg in enumerate(k_segments):
                    if s < len(segments):
                        segments[s][k] = k_seg
                    else:
                        segments.append({k: k_seg})

        return segments
    
    def split_tensor(self, tensor):
        align = self.rmt_config.get('segment_alignment')
        segment_size = self.rmt_config.get('segment_size')
        if align in {'left', None}:
            split_inds = list(range(0, tensor.shape[1], segment_size)) + [tensor.shape[1]]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align in {'right', None}:
            split_inds = (list(range(tensor.shape[1], 0, -segment_size)) + [0])[::-1]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align == 'center':
            n_seg = math.ceil(tensor.shape[1] / segment_size)
            segments = torch.chunk(tensor, n_seg, dim=1)
        else:
            raise NotImplementedError
        return segments

    def process_outputs(self, cell_outputs, **kwargs):
        out = CausalLMOutputWithCrossAttentions()
        full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
        full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

        labels = kwargs.get('labels')
        if labels is not None:
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = full_logits[..., :-1, :].contiguous()
            flat_labels = shift_labels.view(-1)
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            
            loss_fct = CrossEntropyLoss()
            labels_mask = kwargs.get('labels_mask')
            if labels_mask is not None:
                shift_mask = labels_mask[..., :-1].contiguous()

                flat_labels = flat_labels[shift_mask.view(-1)]
                flat_logits = flat_logits[shift_mask.view(-1)]
                
            out['loss'] = loss_fct(flat_logits, flat_labels)

        out['logits'] = full_logits
        segment_keys = ['loss', 'logits']
        if kwargs.get('output_attentions'):
            segment_keys.append('attentions')
        if kwargs.get('output_hidden_states'):
            segment_keys.append('hidden_states')
            out['hidden_states'] = full_hidden_states

        # for seg_num, o in enumerate(cell_outputs):
        #     for key, value in o.items():
        #         if any([sk in key for sk in segment_keys]):
        #             out[f'{key}_{seg_num}'] = value

        return out 
        
    def manage_gradients(self, memory_state, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
            or seg_num + k2 > max_n_segments:
                return True
        
        memory_state = memory_state.detach()
        return False

In [ ]:

num_mem_tokens = 10
# device = torch.device(3)
device = 'cpu'

rmt_config = {'num_mem_tokens': 10, 
               #  'max_n_segments': 1,
               #  'segment_alignment': 'right',
               #  'tokenizer': tokenizer,
               #  'memory_layers': 'all', 
               #  'share_memory_layers': True,
               #  'reconstruction_loss_coef': 0.1,
               #  'k1': -1, 'k2': 3,
               #  'segment_ordering': 'regular',
               #  'input_size': 1024, 
               #  'bptt_depth': -1, 
               #  'sum_loss': False,
             }

base_model = AutoModelForCausalLM.from_pretrained(model_name)
cell = MemoryCell(base_model, num_mem_tokens=2)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
rmt = RecurrentWrapper(cell, max_n_segments=5, segment_size=124, segment_alignment='center')

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch.pop('labels_mask')
# batch.pop('labels')
1

1

In [ ]:
rmt_out = rmt(**batch)

In [ ]:
rmt_out.keys()

odict_keys(['loss', 'logits'])

# Contract-NLI

In [ ]:
from datasets import Dataset

class CNLIDataset(Dataset):
    def __init__(self, json_path):
        with open(json_path, 'r') as f:
            self.samples = json.load(f)

    def __getitem__(self, item):
        return self.samples[item]
    
    def __len__(self):
        return len(self.samples)

In [ ]:
dataset = CNLIDataset('/cephfs/home/bulatov/bulatov/datasets/contract_nli/contract-nli/test_processed.json')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0032s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0160s.) Setting batch_size=4.
[Parallel(n_jobs=12)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 123 out of 123 | elapsed:    0.1s finished


In [ ]:
args.input_seq_len

512

In [ ]:
from torch.nn.utils.rnn import pad_sequence

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]
id_pad_value = tokenizer.eos_token_id

def collate_fn(batch):
    inputs = [b['context'][:args.input_seq_len * 10] for b in batch]
    labels = [b['answer'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs, labels_mask = [], []
    for inp, lab in zip(inputs['input_ids'], labels['input_ids']):
        inp = inp[:args.input_seq_len - len(lab) - 1]
        full_inputs.append(torch.tensor(inp + [gen_token] + lab))
        labels_mask.append(torch.tensor([False] * len(inp) + [True] * (len(lab) + 1)))

    full_inputs = pad_sequence(full_inputs, padding_value=id_pad_value).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T

    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != id_pad_value
    return collated

In [ ]:
batch = collate_fn([dataset[i] for i in range(4)])
# batch = [dataset[i] for i in range(4)]

In [ ]:
segmented = rmt.segment(**batch)

In [ ]:
[s['input_ids'].shape for s in segmented]

[torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 100])]

In [ ]:
labels = batch.pop('labels')
rmt_out = rmt(**batch)

In [ ]:
rmt_out.logits.shape

torch.Size([4, 512, 50258])

In [ ]:
rmt_out.logits[batch['labels_mask']].shape

torch.Size([20, 50258])

In [ ]:
labels[batch['labels_mask']].shape

torch.Size([20])

In [ ]:
tokenizer.decode([ 3673,    44,  1463,   276, 50256])

'NotMentioned<|endoftext|>'

In [ ]:
tokenizer.decode([14539,   603,   434])

'Entailment'

In [ ]:
# rmt_out = rmt(**batch)

self = rmt
input_ids = batch['input_ids']

memory_state = None
segmented = self.segment(input_ids=input_ids, inputs_embeds=None, attention_mask=None)

cell_outputs = []
for seg_num, segment in enumerate(segmented):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)#**batch)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

# out = self.process_outputs(cell_outputs)

In [ ]:
input_ids.shape

torch.Size([2, 248])

In [ ]:
torch.chunk(torch.ones(10), 3)

(tensor([1., 1., 1., 1.]), tensor([1., 1., 1., 1.]), tensor([1., 1.]))

### seq2seq

In [ ]:
from huggingface_hub import hf_hub_download
# scrolls_metric_path = hf_hub_download(repo_id="datasets/tau/scrolls", filename="metrics/scrolls.py")tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]

rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))

# gen_token = tokenizer.eos_token_id

Embedding(50258, 768)

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [ ]:
global_attention_first_token = False  # should be True for LED
encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}
# generate_kwargs = {'max_length': args.targettokenizer.add_tokens('[GEN]', special_tokens=True)
# gen_token = tokenizer.encode('[GEN]')[0]

# rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))_seq_len, 'min_length': args.target_seq_len}
generate_kwargs = {}

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
    labels = [b['output'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs = [torch.tensor(i[:input_size - len(l) - 1] + [gen_token] + l) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
    full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id).T
    
    labels_mask = torch.zeros_like(full_inputs).bool()
    for i, l in enumerate(labels['input_ids']):
        labels_mask[i, -len(l) -1:] = True
    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != tokenizer.pad_token_id

    return collated
    

In [ ]:
seq2seq_task_name = 'quality'
dataset = datasets.load_dataset('tau/scrolls', seq2seq_task_name)
train_dataset = dataset['train']
# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedSampler(train_dataset, shuffle=True, drop_last=False, seed=args.seed)
kwargs = {'pin_memory': True}
train_dataloader = DataLoader(train_dataset, batch_size=2,
                                collate_fn=collate_fn, **kwargs)

Found cached dataset scrolls (/home/bulatov/.cache/huggingface/datasets/tau___scrolls/quality/1.0.0/672021d5d8e1edff998a6ea7a5bff35fdfd0ae243e7cf6a8c88a57a04afb46ac)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)

In [ ]:
batch.keys()

dict_keys(['input_ids', 'labels', 'labels_mask', 'attention_mask'])

In [ ]:
batch['input_ids'].shape

torch.Size([2, 128])

In [ ]:
batch['labels'].shape

torch.Size([2, 128])

In [ ]:
tokenizer.batch_decode(batch['input_ids'])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others to[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\n[GEN]He still wears glasses and value objects like the g

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['labels_mask'])])

['[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 '[GEN]He still wears glasses and value objects like the gold watch given to Si.']

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['attention_mask'])])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others toTraining new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\nHe still wears glasses and value objects like the gold watch 

In [ ]:
out = rmt(**batch)

In [ ]:
out.loss

tensor(60.2067, grad_fn=<NllLossBackward>)

In [ ]:
out.loss

tensor(61.4377, grad_fn=<NllLossBackward>)